# VacationPy
---

## Code to Import Libraries and Load the Weather and Coordinates Data

In [21]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
#import seaborn as sns

# Import API key
from api_keys import geoapify_key

In [23]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,-15.70,88,100,7.07,RU,1738732849
1,1,taksimo,56.3444,114.8794,-5.08,73,100,3.40,RU,1738732850
2,2,portland,45.5234,-122.6762,37.65,83,100,3.00,US,1738732574
3,3,port-aux-francais,-49.3500,70.2167,42.24,96,93,24.11,TF,1738732852
4,4,alice springs,-23.7000,133.8833,98.15,20,20,25.32,AU,1738732853


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
# Configure the map plot
coordinates = {
    "Lat": (city_data_df["Lat"]),
    "Lng": (city_data_df["Lng"]),
    "City": (city_data_df["City"]),
    "Humidity": (city_data_df["Humidity"])
}
coordinates_df = pd.DataFrame(coordinates)

map_plot = coordinates_df.hvplot.points(
                                            x = "Lng",
                                            y = "Lat",
                                            geo = True,
                                            size = "Humidity",  # Size of the points based on humidity
                                            color = "City",  
                                            xlabel = "x",
                                            ylabel = "y",
                                            alpha = 0.6,
                                            tiles = "OSM"    # Using OpenStreetMap as the background
                                      )
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [29]:
# Narrow down cities that fit criteria and drop any results with null values
my_city_df = city_data_df.loc[(city_data_df["Max Temp"] <= 33) & (city_data_df["Max Temp"] >= 18) & (city_data_df["Wind Speed"] <= 10)].dropna()

# Display sample data
my_city_df.head(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,bethel,41.3712,-73.4140,26.56,65,36,1.99,US,1738732869
25,25,albany,42.6001,-73.9662,19.54,74,11,9.37,US,1738732882
53,53,jamestown,42.0970,-79.2353,18.55,79,100,8.05,US,1738732919
62,62,kyshtovka,56.5550,76.6271,20.89,96,98,5.32,RU,1738732929
70,70,berdyansk,46.7503,36.7944,27.84,86,11,6.69,UA,1738732939


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
# Create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_city_df.loc[:,["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(5)

,City,Country,Lat,Lng,Humidity,Hotel Name
16,bethel,US,41.3712,-73.4140,65,
25,albany,US,42.6001,-73.9662,74,
53,jamestown,US,42.0970,-79.2353,79,
62,kyshtovka,RU,56.5550,76.6271,96,
70,berdyansk,UA,46.7503,36.7944,86,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set parameters to search for a hotel
radius = 10000
params = {
            "categories": "accommodation.hotel",
            "apiKey": geoapify_key
        }
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]

    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
      
    # Log the search results
    print(f'{hotel_df.loc[index, "City"]} - nearest hotel: {hotel_df.loc[index, "Hotel Name"]}')

# Display sample data
hotel_df.head(10)

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
albany - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
kyshtovka - nearest hotel: No hotel found
berdyansk - nearest hotel: No hotel found
ketchikan - nearest hotel: Cape Fox Lodge
kodiak - nearest hotel: No hotel found
bingol - nearest hotel: Grand Hotel Berti
nizhnesortymskiy - nearest hotel: Нижнесортымский
saint-francois - nearest hotel: Hôtel Epona
karasuk - nearest hotel: Фаворит
grand falls-windsor - nearest hotel: Hotel Robin Hood
khot'kovo - nearest hotel: Адамас
hami - nearest hotel: Хами
turtas - nearest hotel: Сибирь
palmer - nearest hotel: No hotel found
nako - nearest hotel: Himalayan Paradise Hotel and restaurant
chomutov - nearest hotel: Royal
chatellerault - nearest hotel: Ibis
furstenfeld - nearest hotel: Heuberger
kuqa - nearest hotel: Kuqa Hotel - foreigners accepted
goynuk - nearest hotel: No hotel found
balaklava - nearest hotel: Рыбацкая слобода
jogeva - nearest hote

,City,Country,Lat,Lng,Humidity,Hotel Name
16,bethel,US,41.3712,-73.4140,65,Hampton Inn Danbury
25,albany,US,42.6001,-73.9662,74,No hotel found
53,jamestown,US,42.0970,-79.2353,79,DoubleTree Jamestown
62,kyshtovka,RU,56.5550,76.6271,96,No hotel found
70,berdyansk,UA,46.7503,36.7944,86,No hotel found
92,ketchikan,US,55.3422,-131.6461,28,Cape Fox Lodge
106,kodiak,US,57.7900,-152.4072,93,No hotel found
138,bingol,TR,38.8847,40.4939,80,Grand Hotel Berti
198,nizhnesortymskiy,RU,62.4397,71.7603,95,Нижнесортымский
273,saint-francois,FR,46.4154,3.9054,100,Hôtel Epona


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
# Configure the map plot
coordinates = {
    "Lat": hotel_df["Lat"],
    "Lng": hotel_df["Lng"],
    "City": hotel_df["City"],
    "Humidity": hotel_df["Humidity"],
    "Hotel Name": hotel_df["Hotel Name"],
    "Country": hotel_df["Country"]
}
coordinates_df = pd.DataFrame(coordinates)

map_plot = coordinates_df.hvplot.points(
                                            x = "Lng",
                                            y = "Lat",
                                            geo = True,
                                            size = "Humidity",  # Size of the points based on humidity
                                            color = "City",  
                                            xlabel = "x",
                                            ylabel = "y",
                                            alpha = 0.6,
                                            tiles = "OSM",    # Using OpenStreetMap as the background
                                            hover_cols = ["Hotel Name", "Country"]
                                      )

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)